## Joel Brigida

## CDA4321: Cryptographic Engineering

## Prelab 1

## 9/6/2023

In [1]:
a = 2^30 - 18                   # Number to be reduced
p1 = 2^17 - 1                   # Modulus 1 Mersenne Prime
p2 = 2^26 - 5                   # Modulus 2 Pseudo-Mersenne Prime
b = 2^16                        # A power of 2

# Question 1

## Question 1a) $$ a \bmod p_1 $$

In [2]:
ah = a >> 17                    # resulting operation is 13 bits
al = p1 & a                     # need upper p1 as a mask: p1 = 0x1ffff (17 1's)

print(hex(2^17 - 1))
print(hex(2^30 - 1), '\n')

print('a = 2^30 - 18 = ', hex(a))
print('p1 = 2^17 - 1 = ', hex(p1), '\n')

print('ah: (high 13 bits) ', hex(ah))
print('al: (lower 17 bits) ', hex(al), '\n')

# print(f'hex(ah + al) = {hex(ah + al)}')

r = ah + al                     # Add the lower and upper bits together
print(f'r = ah + al = {r.nbits()} bits = {hex(r)}')
print(f'r > p1 ? {r > p1}\n')

# Still at 18 bits, need to get to 17 bits ... what can I remove?

r -= p1                         # subtract the 17 bit modulus from r
print(f'r -= p1 = {hex(r)}')
print(f'r > p1 ? {r > p1}\n')

print(f'Residue of a % p1 = {hex(r)}')
print(f'Answer matches correct result ? {hex(r) == hex(a % p1)}')

0x1ffff
0x3fffffff 

a = 2^30 - 18 =  0x3fffffee
p1 = 2^17 - 1 =  0x1ffff 

ah: (high 13 bits)  0x1fff
al: (lower 17 bits)  0x1ffee 

r = ah + al = 18 bits = 0x21fed
r > p1 ? True

r -= p1 = 0x1fee
r > p1 ? False

Residue of a % p1 = 0x1fee
Answer matches correct result ? True


## Question 1b) $$ a \bmod p2 $$

In [3]:
ah = a >> 26

al = a & (2^26 - 1)
ah = a >> 26                  # High Bits: Result is 4 bits 0xF
print(f'ah = {hex(ah)}')

al = a & (p2 + 4)             # Low bits: Lowest 26 bits of dividend: p2 + 4 = 26 1's
print(f'al2 = {hex(al)}')

r = (ah * 5) + al            # multiply by 5 because of the -5 in: p2 = 2^26 - 5
print(f'r = {hex(r)} (before while loop)\n')

while(r >= p2):
    r = r - p2

print(f'r: Residue of {hex(a)} % {hex(p2)} = {hex(r)}\n')

ah = 0xf
al2 = 0x3ffffee
r = 0x4000039 (before while loop)

r: Residue of 0x3fffffee % 0x3fffffb = 0x3e



## Question 1c) $$ a \bmod b $$

In [4]:
a = 2^30 - 18           # Number to be reduced
b = 2^16                # A power of 2

print(f'2^16     = {bin(2^16)}')
print(f'2^16 - 1  = {bin(2^16 - 1)}\n')

c = a & (2^16 - 1)      # Mask 'a' == (a % 2^16)

print(f'Residue of a % b = {hex(c)}\n')
print(f'Answer matches correct result ? {hex(c) == hex(a % b)}')

2^16     = 0b10000000000000000
2^16 - 1  = 0b1111111111111111

Residue of a % b = 0xffee

Answer matches correct result ? True


# Question 2

## Question 2a) FLT: Constant time regardless of input (depends on exponent) $$ a^{p-2} \bmod p \equiv a^{-1} \bmod p $$

In [5]:
# FLT = a^(p-2) % p === a^(-1) % p
a = 51                  # modulus

print(hex(a^(p1-2) % p1), '\n')
print(hex(a^(-1) % p1), '\n')

print(a^(p1-2) % p1 == (a^(-1) % p1), '\n') 

# need n - 1 multiplications for a^n
# a^4 = a * a * a * a (4 multiplications)
# a^5 = a * a * a * a * a (5 multiplications)
a = 51
r = a

for i in range(p1 - 3):
    r = (r * a) % p1

print(hex(r), '\n')
print(f'Total Iterations: {p1 - 3}\n')

0x1f5f5 

0x1f5f5 

True 

0x1f5f5 

Total Iterations: 131068



## Question 2b) EEA

In [6]:
def EEA(b, p1, count):
    # p1 = 0x0001FFFF
    a = p1
    # b = 51                                  # modulus
    u = 1
    v = 0
    d = a
    v1 = 0
    v3 = b

    while(v3 != 0):
        q = d // v3;                        # integer division
        t3 = d % v3;
        t1 = u - (q * v1);
        
        if(t1 < 0):
            print(f'While: t1 = {hex(t1)} = {t1}\n')
            t1 = t1 + p1
        
        u = v1
        d = v3
        v1 = t1
        v3 = t3
        count += 1


    v = (d - (a * u)) / b;                  # can be negative

    if(v < 0):                              # if negative: Add the modulus
        v = v + a

    print(f'v = {hex(v)}')
    return count

b = 51                                      # modulus
p1 = 0x0001FFFF

counter = EEA(b, p1, 0)

print(f'Iterations: {counter}')

While: t1 = -0x33 = -51

v = 0x1f5f5
Iterations: 2


# Problem 3:

## 3a) How many loop iterations does it take to compute the output for each method for $a = 51$ (exercise 2)?

### a) FLT: Total Iterations: 131068
### b) EEA: Total Iterations: 2

## 3b) Which Method is faster, in general? Why?

### EEA is in general, faster because it has time complexity: $$ O(log^{2}(p)) $$
### FLT is slower with time complexity $$ O(log^{3}(p)) $$

## 3c) Which Method is Constant Time? Why?

### Constant Time = number of iterations is the same independent of the input.

### EEA is constant time regardless of input because the algorithm is only dependant on the exponent used.
